# Text Classification Model for [PROJECT NAME] Using TensorFlow Version 1
### David Lowe
### January 1, 2021

Template Credit: Adapted from a template made available by Dr. Jason Brownlee of Machine Learning Mastery. [https://machinelearningmastery.com/]

SUMMARY: This project aims to construct a text classification model using a neural network and document the end-to-end steps using a template. The [PROJECT NAME] dataset is a binary classification situation where we attempt to predict one of the two possible outcomes.

INTRODUCTION: [SAMPLE PARAGRAPH: The Movie Review Data is a collection of movie reviews retrieved from the imdb.com website in the early 2000s by Bo Pang and Lillian Lee. The reviews were collected and made available as part of their research on natural language processing. The dataset comprises 1,000 positive and 1,000 negative movie reviews drawn from an archive of the rec.arts.movies.reviews newsgroup hosted at IMDB. We will use the last 100 positive reviews and the last 100 negative reviews as a test set (200 reviews) and the remaining 1,800 reviews as the training dataset. We will construct a bag-of-words model and analyzed it with a simple TensorFlow deep learning network.]

ANALYSIS: [SAMPLE PARAGRAPH: In this modeling iteration, the baseline model's performance achieved an average accuracy score of 86.06% after 25 epochs with ten iterations of cross-validation. Furthermore, the final model processed the test dataset with an accuracy measurement of 86.50%.]

CONCLUSION: In this modeling iteration, the bag-of-words TensorFlow model appeared to be suitable for modeling this dataset. We should consider experimenting with TensorFlow for further modeling.

Dataset Used: [PROJECT NAME]

Dataset ML Model: Binary class text classification with text-oriented features

Dataset Reference: [http://www.cs.cornell.edu/people/pabo/movie-review-data/review_polarity.tar.gz]

One potential source of performance benchmarks: [https://machinelearningmastery.com/deep-learning-bag-of-words-model-sentiment-analysis/]

A deep-learning text classification project generally can be broken down into five major tasks:

1. Prepare Environment
2. Load and Prepare Text Data
3. Define and Train Models
4. Evaluate and Optimize Models
5. Finalize Model and Make Predictions

# Task 1 - Prepare Environment

In [1]:
# # Install the packages to support accessing environment variable and SQL databases
# !pip install python-dotenv PyMySQL boto3

In [2]:
# # Retrieve GPU configuration information from Colab
# gpu_info = !nvidia-smi
# gpu_info = '\n'.join(gpu_info)
# if gpu_info.find('failed') >= 0:
#     print('Select the Runtime → "Change runtime type" menu to enable a GPU accelerator, ')
#     print('and then re-execute this cell.')
# else:
#     print(gpu_info)

In [3]:
# # Retrieve memory configuration information from Colab
# from psutil import virtual_memory
# ram_gb = virtual_memory().total / 1e9
# print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

# if ram_gb < 20:
#     print('To enable a high-RAM runtime, select the Runtime → "Change runtime type"')
#     print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
#     print('re-execute this cell.')
# else:
#     print('You are using a high-RAM runtime!')

In [4]:
# Retrieve CPU information from the system
ncpu = !nproc
print("The number of available CPUs is:", ncpu[0])

The number of available CPUs is: 4


## 1.a) Load libraries and modules

In [5]:
# Set the random seed number for reproducible results
seedNum = 888

In [6]:
# Load libraries and packages
import random
random.seed(seedNum)
import numpy as np
np.random.seed(seedNum)
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
import sys
import boto3
import shutil
import string
import nltk
from nltk.corpus import stopwords
from collections import Counter
from datetime import datetime
from dotenv import load_dotenv
from sklearn.model_selection import RepeatedKFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import tensorflow as tf
tf.random.set_seed(seedNum)
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import ReduceLROnPlateau

In [7]:
nltk.download('popular')

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     /home/pythonml/nltk_data...
[nltk_data]    |   Unzipping corpora/cmudict.zip.
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     /home/pythonml/nltk_data...
[nltk_data]    |   Unzipping corpora/gazetteers.zip.
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     /home/pythonml/nltk_data...
[nltk_data]    |   Unzipping corpora/genesis.zip.
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     /home/pythonml/nltk_data...
[nltk_data]    |   Unzipping corpora/gutenberg.zip.
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     /home/pythonml/nltk_data...
[nltk_data]    |   Unzipping corpora/inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /home/pythonml/nltk_data...
[nltk_data]    |   Unzipping corpora/movie_reviews.zip.
[nltk_data]    | Downloading 

True

## 1.b) Set up the controlling parameters and functions

In [8]:
# Begin the timer for the script processing
startTimeScript = datetime.now()

# Set up the number of CPU cores available for multi-thread processing
n_jobs = 1

# Set up the flag to stop sending progress emails (setting to True will send status emails!)
notifyStatus = False

# Set Pandas options
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 140)

# Set the percentage sizes for splitting the dataset
test_set_size = 0.2
val_set_size = 0.25

# Set the number of folds for cross validation
n_folds = 5
n_iterations = 2

# Set various default modeling parameters
default_loss = 'binary_crossentropy'
default_metrics = ['accuracy']
default_optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
default_kernel_init = tf.keras.initializers.GlorotUniform(seed=seedNum)
default_epochs = 25
default_batch_size = 16

# Define the labels to use for graphing the data
train_metric = "accuracy"
validation_metric = "val_accuracy"
train_loss = "loss"
validation_loss = "val_loss"

# Check the number of GPUs accessible through TensorFlow
print('Num GPUs Available:', len(tf.config.list_physical_devices('GPU')))

# Print out the TensorFlow version for confirmation
print('TensorFlow version:', tf.__version__)

Num GPUs Available: 0
TensorFlow version: 2.3.1


In [9]:
# Set up the parent directory location for loading the dotenv files
# from google.colab import drive
# drive.mount('/content/gdrive')
# gdrivePrefix = '/content/gdrive/My Drive/Colab_Downloads/'
# env_path = '/content/gdrive/My Drive/Colab Notebooks/'
# dotenv_path = env_path + "python_script.env"
# load_dotenv(dotenv_path=dotenv_path)

# Set up the dotenv file for retrieving environment variables
# env_path = "/Users/david/PycharmProjects/"
# dotenv_path = env_path + "python_script.env"
# load_dotenv(dotenv_path=dotenv_path)

In [10]:
# Set up the email notification function
def status_notify(msg_text):
    access_key = os.environ.get('SNS_ACCESS_KEY')
    secret_key = os.environ.get('SNS_SECRET_KEY')
    aws_region = os.environ.get('SNS_AWS_REGION')
    topic_arn = os.environ.get('SNS_TOPIC_ARN')
    if (access_key is None) or (secret_key is None) or (aws_region is None):
        sys.exit("Incomplete notification setup info. Script Processing Aborted!!!")
    sns = boto3.client('sns', aws_access_key_id=access_key, aws_secret_access_key=secret_key, region_name=aws_region)
    response = sns.publish(TopicArn=topic_arn, Message=msg_text)
    if response['ResponseMetadata']['HTTPStatusCode'] != 200 :
        print('Status notification not OK with HTTP status code:', response['ResponseMetadata']['HTTPStatusCode'])

In [11]:
if notifyStatus: status_notify('(TensorFlow Text Classification) Task 1 - Prepare Environment has begun on ' + datetime.now().strftime('%A %B %d, %Y %I:%M:%S %p'))

In [12]:
# Reset the random number generators
def reset_random(x):
    random.seed(x)
    np.random.seed(x)
    tf.random.set_seed(x)

In [13]:
if notifyStatus: status_notify('(TensorFlow Text Classification) Task 1 - Prepare Environment completed on ' + datetime.now().strftime('%A %B %d, %Y %I:%M:%S %p'))

# Task 2 - Load and Prepare Text Data

In [14]:
if notifyStatus: status_notify('(TensorFlow Text Classification) Task 2 - Load and Prepare Text Data has begun on ' + datetime.now().strftime('%A %B %d, %Y %I:%M:%S %p'))

## 2.a) Download Text Data Archive

In [15]:
# Clean up the old files and download directories before receiving new ones
!rm -rf staging/
!rm review_polarity.tar.gz
!rm vocabulary.txt

rm: cannot remove 'review_polarity.tar.gz': No such file or directory
rm: cannot remove 'vocabulary.txt': No such file or directory


In [16]:
!wget https://dainesanalytics.com/datasets/cornell-movie-review-polarity/review_polarity.tar.gz

--2020-12-20 19:09:59--  https://dainesanalytics.com/datasets/cornell-movie-review-polarity/review_polarity.tar.gz
Resolving dainesanalytics.com (dainesanalytics.com)... 13.225.146.37, 13.225.146.126, 13.225.146.46, ...
Connecting to dainesanalytics.com (dainesanalytics.com)|13.225.146.37|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3127238 (3.0M) [application/x-gzip]
Saving to: ‘review_polarity.tar.gz’

review_polarity.tar 100%[===================>]   2.98M  14.8MB/s    in 0.2s    

2020-12-20 19:10:00 (14.8 MB/s) - ‘review_polarity.tar.gz’ saved [3127238/3127238]



## 2.b) Splitting Data for Training and Validation

In [17]:
staging_dir = 'staging/'
!mkdir staging/
!mkdir staging/testing/
!mkdir staging/testing/pos/
!mkdir staging/testing/neg/

In [18]:
local_archive = 'review_polarity.tar.gz'
shutil.unpack_archive(local_archive, staging_dir)

In [19]:
training_dir = 'staging/txt_sentoken/'
testing_dir = 'staging/testing/'
classA_name = 'pos'
classB_name = 'neg'

In [20]:
# Brief listing of training text files for both classes before splitting
training_classA_dir = os.path.join(training_dir, classA_name)
training_classA_files = os.listdir(training_classA_dir)
print('Number of training images for', classA_name, ':', len(training_classA_files))
print('Training samples for', classA_name, ':', training_classA_files[:10])

training_classB_dir = os.path.join(training_dir, classB_name)
training_classB_files = os.listdir(training_classB_dir)
print('Number of training images for', classB_name, ':', len(training_classB_files))
print('Training samples for', classB_name, ':', training_classB_files[:10])

Number of training images for pos : 1000
Training samples for pos : ['cv000_29590.txt', 'cv001_18431.txt', 'cv002_15918.txt', 'cv003_11664.txt', 'cv004_11636.txt', 'cv005_29443.txt', 'cv006_15448.txt', 'cv007_4968.txt', 'cv008_29435.txt', 'cv009_29592.txt']
Number of training images for neg : 1000
Training samples for neg : ['cv000_29416.txt', 'cv001_19502.txt', 'cv002_17424.txt', 'cv003_12683.txt', 'cv004_12641.txt', 'cv005_29357.txt', 'cv006_17022.txt', 'cv007_4992.txt', 'cv008_29326.txt', 'cv009_29417.txt']


In [21]:
# Move the testing files from the training directories
testing_classA_dir = os.path.join(testing_dir, classA_name)
testing_classB_dir = os.path.join(testing_dir, classB_name)
file_prefix = 'cv9'

i = 0
for text_file in training_classA_files:
    if text_file.startswith(file_prefix):
        source_file = training_classA_dir + '/' + text_file
        dest_file = testing_classA_dir + '/' + text_file
        print('Moving file from', source_file, 'to', dest_file)
        shutil.move(source_file, dest_file)
        i = i + 1
print('Number of', classA_name, 'files moved:', i, '\n')

i = 0
for text_file in training_classB_files:
    if text_file.startswith(file_prefix):
        source_file = training_classB_dir + '/' + text_file
        dest_file = testing_classB_dir + '/' + text_file
        print('Moving file from', source_file, 'to', dest_file)
        shutil.move(source_file, dest_file)
        i = i + 1
print('Number of', classB_name, 'files moved:', i)

Moving file from staging/txt_sentoken/pos/cv900_10331.txt to staging/testing/pos/cv900_10331.txt
Moving file from staging/txt_sentoken/pos/cv901_11017.txt to staging/testing/pos/cv901_11017.txt
Moving file from staging/txt_sentoken/pos/cv902_12256.txt to staging/testing/pos/cv902_12256.txt
Moving file from staging/txt_sentoken/pos/cv903_17822.txt to staging/testing/pos/cv903_17822.txt
Moving file from staging/txt_sentoken/pos/cv904_24353.txt to staging/testing/pos/cv904_24353.txt
Moving file from staging/txt_sentoken/pos/cv905_29114.txt to staging/testing/pos/cv905_29114.txt
Moving file from staging/txt_sentoken/pos/cv906_11491.txt to staging/testing/pos/cv906_11491.txt
Moving file from staging/txt_sentoken/pos/cv907_3541.txt to staging/testing/pos/cv907_3541.txt
Moving file from staging/txt_sentoken/pos/cv908_16009.txt to staging/testing/pos/cv908_16009.txt
Moving file from staging/txt_sentoken/pos/cv909_9960.txt to staging/testing/pos/cv909_9960.txt
Moving file from staging/txt_sento

In [22]:
# Brief listing of training text files for both classes after splitting
training_classA_files = os.listdir(training_classA_dir)
print('Number of training files for', classA_name, ':', len(training_classA_files))
print('Training samples for', classA_name, ':', training_classA_files[:10])

training_classB_files = os.listdir(training_classB_dir)
print('Number of training files for', classB_name, ':', len(training_classB_files))
print('Training samples for', classB_name, ':', training_classB_files[:10])

Number of training files for pos : 900
Training samples for pos : ['cv000_29590.txt', 'cv001_18431.txt', 'cv002_15918.txt', 'cv003_11664.txt', 'cv004_11636.txt', 'cv005_29443.txt', 'cv006_15448.txt', 'cv007_4968.txt', 'cv008_29435.txt', 'cv009_29592.txt']
Number of training files for neg : 900
Training samples for neg : ['cv000_29416.txt', 'cv001_19502.txt', 'cv002_17424.txt', 'cv003_12683.txt', 'cv004_12641.txt', 'cv005_29357.txt', 'cv006_17022.txt', 'cv007_4992.txt', 'cv008_29326.txt', 'cv009_29417.txt']


In [23]:
# Brief listing of testing text files for both classes after splitting
testing_classA_files = os.listdir(testing_classA_dir)
print('Number of testing files for', classA_name, ':', len(testing_classA_files))
print('Test samples for', classA_name, ':', testing_classA_files[:10])

testing_classB_files = os.listdir(testing_classB_dir)
print('Number of testing files for', classB_name, ':', len(testing_classB_files))
print('Test samples for', classB_name, ':', testing_classB_files[:10])

Number of testing files for pos : 100
Test samples for pos : ['cv900_10331.txt', 'cv901_11017.txt', 'cv902_12256.txt', 'cv903_17822.txt', 'cv904_24353.txt', 'cv905_29114.txt', 'cv906_11491.txt', 'cv907_3541.txt', 'cv908_16009.txt', 'cv909_9960.txt']
Number of testing files for neg : 100
Test samples for neg : ['cv900_10800.txt', 'cv901_11934.txt', 'cv902_13217.txt', 'cv903_18981.txt', 'cv904_25663.txt', 'cv905_28965.txt', 'cv906_12332.txt', 'cv907_3193.txt', 'cv908_17779.txt', 'cv909_9973.txt']


## 2.c) Load Document and Build Vocabulary

In [24]:
# load doc into memory
def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text

In [25]:
# turn a doc into clean tokens
def clean_doc(doc):
    # split into tokens by white space
    tokens = doc.split()
    # remove punctuation from each token
    table = str.maketrans('', '', string.punctuation)
    tokens = [w.translate(table) for w in tokens]
    # remove remaining tokens that are not alphabetic
    tokens = [word for word in tokens if word.isalpha()]
    # filter out stop words
    stop_words = set(stopwords.words('english'))
    tokens = [w for w in tokens if not w in stop_words]
    # filter out short tokens
    tokens = [word for word in tokens if len(word) > 1]
    return tokens

In [26]:
# load doc and add to vocab
def add_doc_to_vocab(filename, vocab):
    # load doc
    doc = load_doc(filename)
    # clean doc
    tokens = clean_doc(doc)
    # update counts
    vocab.update(tokens)

In [27]:
# load all docs in a directory
def build_vocab_from_docs(directory, vocab):
    # walk through all files in the folder
    i = 0
    print('Processing the text files and showing the first 10...')
    for filename in os.listdir(directory):
        # skip files that do not have the right extension
        if not filename.endswith(".txt"):
            continue
        # create the full path of the file to open
        path = directory + '/' + filename
        # add doc to vocab
        add_doc_to_vocab(path, vocab)
        i = i + 1
        if i < 10: print('Loaded %s' % path)
    print('Total number of text files loaded:', i, '\n')

In [28]:
# define vocab
vocab = Counter()
# add all docs to vocab
build_vocab_from_docs(training_classA_dir, vocab)
build_vocab_from_docs(training_classB_dir, vocab)
# print the size of the vocab
print('The total number of words in the vocabulary:', len(vocab))
# print the top words in the vocab
top_words = 50
print('The top', top_words, 'words in the vocabulary:\n', vocab.most_common(top_words))

Processing the text files and showing the first 10...
Loaded staging/txt_sentoken/pos/cv000_29590.txt
Loaded staging/txt_sentoken/pos/cv001_18431.txt
Loaded staging/txt_sentoken/pos/cv002_15918.txt
Loaded staging/txt_sentoken/pos/cv003_11664.txt
Loaded staging/txt_sentoken/pos/cv004_11636.txt
Loaded staging/txt_sentoken/pos/cv005_29443.txt
Loaded staging/txt_sentoken/pos/cv006_15448.txt
Loaded staging/txt_sentoken/pos/cv007_4968.txt
Loaded staging/txt_sentoken/pos/cv008_29435.txt
Total number of text files loaded: 900 

Processing the text files and showing the first 10...
Loaded staging/txt_sentoken/neg/cv000_29416.txt
Loaded staging/txt_sentoken/neg/cv001_19502.txt
Loaded staging/txt_sentoken/neg/cv002_17424.txt
Loaded staging/txt_sentoken/neg/cv003_12683.txt
Loaded staging/txt_sentoken/neg/cv004_12641.txt
Loaded staging/txt_sentoken/neg/cv005_29357.txt
Loaded staging/txt_sentoken/neg/cv006_17022.txt
Loaded staging/txt_sentoken/neg/cv007_4992.txt
Loaded staging/txt_sentoken/neg/cv008

In [29]:
# keep tokens with a min occurrence
min_occurane = 2
tokens = [k for k,c in vocab.items() if c >= min_occurane]
print('The number of words with the minimum appearance:', len(tokens))

The number of words with the minimum appearance: 25767


In [30]:
# save list to file
def save_list(lines, filename):
    # convert lines to a single blob of text
    data = '\n'.join(lines)
    # open file
    file = open(filename, 'w')
    # write text
    file.write(data)
    # close file
    file.close()

# save tokens to a vocabulary file
save_list(tokens, 'vocabulary.txt')

## 2.d) Create Tokenizer and Encode the Input Text

In [31]:
# load doc, clean and return line of tokens
def doc_to_line(filename, vocab):
    # load the doc
    doc = load_doc(filename)
    # clean doc
    tokens = clean_doc(doc)
    # filter by vocab
    tokens = [w for w in tokens if w in vocab]
    return ' '.join(tokens)

In [32]:
# load all docs in a directory
def process_docs_to_lines(directory, vocab):
    lines = list()
    # walk through all files in the folder
    for filename in os.listdir(directory):
        # create the full path of the file to open
        path = directory + '/' + filename
        # load and clean the doc
        line = doc_to_line(path, vocab)
        # add to list
        lines.append(line)
    return lines

In [33]:
# load the vocabulary
vocab_filename = 'vocabulary.txt'
vocab = load_doc(vocab_filename)
vocab = vocab.split()
vocab = set(vocab)
print('Number of tokens in the vocabulary:', len(vocab))

Number of tokens in the vocabulary: 25767


In [34]:
# prepare bag of words encoding of docs
def encode_data(train_docs, val_docs, mode='binary'):
    # create the tokenizer
    tokenizer = Tokenizer()
    # fit the tokenizer on the documents
    tokenizer.fit_on_texts(train_docs)
    # encode training data set
    train_encoded = tokenizer.texts_to_matrix(train_docs, mode=mode)
    # encode validation data set
    val_encoded = tokenizer.texts_to_matrix(val_docs, mode=mode)
    return train_encoded, val_encoded

In [35]:
# Load all training cases
positive_train_cases = process_docs_to_lines(training_classA_dir, vocab)
print('The number of positive reviews processed:', len(positive_train_cases))
negative_train_cases = process_docs_to_lines(training_classB_dir, vocab)
print('The number of negative reviews processed:', len(negative_train_cases))
training_docs =  negative_train_cases + positive_train_cases
y_train = np.array([0 for _ in range(len(negative_train_cases))] + [1 for _ in range(len(positive_train_cases))])
print('The shape of the encoded training classes:', y_train.shape)

The number of positive reviews processed: 900
The number of negative reviews processed: 900
The shape of the encoded training classes: (1800,)


In [36]:
# load all validation cases
positive_test_cases = process_docs_to_lines(testing_classA_dir, vocab)
print('The number of positive reviews processed:', len(positive_test_cases))
negative_test_cases = process_docs_to_lines(testing_classB_dir, vocab)
print('The number of negative reviews processed:', len(negative_test_cases))
testing_docs = negative_test_cases + positive_test_cases
y_test = np.array([0 for _ in range(len(negative_test_cases))] + [1 for _ in range(len(positive_test_cases))])
print('The shape of the encoded test classes:', y_test.shape)

The number of positive reviews processed: 100
The number of negative reviews processed: 100
The shape of the encoded test classes: (200,)


In [37]:
# encode training and validation datasets
X_train, X_test = encode_data(training_docs, testing_docs)
print('The shape of the encoded training dataset:', X_train.shape)
print('The shape of the encoded validation dataset:', X_test.shape)

The shape of the encoded training dataset: (1800, 25768)
The shape of the encoded validation dataset: (200, 25768)


In [38]:
if notifyStatus: status_notify('(TensorFlow Text Classification) Task 2 - Load and Prepare Text Data completed on ' + datetime.now().strftime('%A %B %d, %Y %I:%M:%S %p'))

# Task 3 - Define and Train Models

In [39]:
if notifyStatus: status_notify('(TensorFlow Text Classification) Task 3 - Define and Train Models has begun on ' + datetime.now().strftime('%A %B %d, %Y %I:%M:%S %p'))

In [40]:
# Define the default numbers of input/output for modeling
num_inputs = X_train.shape[1]
num_outputs = 1

In [41]:
# Define the baseline model for benchmarking
def create_nn_model(n_inputs=num_inputs, n_outputs=num_outputs, layer1_nodes=50, layer1_dropout=0, opt_param=default_optimizer, init_param=default_kernel_init):
    nn_model = keras.Sequential([
        keras.layers.Dense(layer1_nodes, input_shape=(n_inputs,), activation='relu', kernel_initializer=init_param),
#         keras.layers.Dropout(layer1_dropout),
        keras.layers.Dense(n_outputs, activation='sigmoid', kernel_initializer=init_param)
    ])
    nn_model.compile(loss=default_loss, optimizer=opt_param, metrics=default_metrics)
    return nn_model

In [42]:
# Initialize the default model and get a baseline result
startTimeModule = datetime.now()
results = list()
iteration = 0
cv = RepeatedKFold(n_splits=n_folds, n_repeats=n_iterations, random_state=seedNum)
for train_ix, val_ix in cv.split(X_train):
    feature_train, feature_validation = X_train[train_ix], X_train[val_ix]
    target_train, target_validation = y_train[train_ix], y_train[val_ix]
    reset_random(seedNum)
    baseline_model = create_nn_model()
    baseline_model.fit(feature_train, target_train, epochs=default_epochs, batch_size=default_batch_size, verbose=0)
    model_metric = baseline_model.evaluate(feature_validation, target_validation, verbose=0)[1]
    iteration = iteration + 1
    print('Accuracy measurement from iteration %d >>> %.2f%%' % (iteration, model_metric*100))
    results.append(model_metric)
validation_score = np.mean(results)
validation_variance = np.std(results)
print('Average model accuracy from all iterations: %.2f%% (%.2f%%)' % (validation_score*100, validation_variance*100))
print('Total time for model fitting and cross validating:', (datetime.now() - startTimeModule))

Accuracy measurement from iteration 1 >>> 87.22%
Accuracy measurement from iteration 2 >>> 84.72%
Accuracy measurement from iteration 3 >>> 85.28%
Accuracy measurement from iteration 4 >>> 83.89%
Accuracy measurement from iteration 5 >>> 86.11%
Accuracy measurement from iteration 6 >>> 86.67%
Accuracy measurement from iteration 7 >>> 83.33%
Accuracy measurement from iteration 8 >>> 86.94%
Accuracy measurement from iteration 9 >>> 87.22%
Accuracy measurement from iteration 10 >>> 86.67%
Average model accuracy from all iterations: 85.81% (1.35%)
Total time for model fitting and cross validating: 0:02:50.603388


In [43]:
if notifyStatus: status_notify('(TensorFlow Text Classification) Task 3 - Define and Train Models completed on ' + datetime.now().strftime('%A %B %d, %Y %I:%M:%S %p'))

# Task 4 - Evaluate and Optimize Models

In [44]:
if notifyStatus: status_notify('(TensorFlow Text Classification) Task 4 - Evaluate and Optimize Models has begun on ' + datetime.now().strftime('%A %B %d, %Y %I:%M:%S %p'))

## 4.a) Alternate Model One

In [45]:
# encode training and validation datasets
X_train, X_test = encode_data(training_docs, testing_docs, 'count')
print('The shape of the encoded training dataset:', X_train.shape)
print('The shape of the encoded validation dataset:', X_test.shape)

The shape of the encoded training dataset: (1800, 25768)
The shape of the encoded validation dataset: (200, 25768)


In [46]:
# Initialize the default model and get a baseline result
startTimeModule = datetime.now()
results = list()
iteration = 0
cv = RepeatedKFold(n_splits=n_folds, n_repeats=n_iterations, random_state=seedNum)
for train_ix, val_ix in cv.split(X_train):
    feature_train, feature_validation = X_train[train_ix], X_train[val_ix]
    target_train, target_validation = y_train[train_ix], y_train[val_ix]
    reset_random(seedNum)
    alternate_model_1 = create_nn_model()
    alternate_model_1.fit(feature_train, target_train, epochs=default_epochs, batch_size=default_batch_size, verbose=0)
    model_metric = alternate_model_1.evaluate(feature_validation, target_validation, verbose=0)[1]
    iteration = iteration + 1
    print('Accuracy measurement from iteration %d >>> %.2f%%' % (iteration, model_metric*100))
    results.append(model_metric)
validation_score = np.mean(results)
validation_variance = np.std(results)
print('Average model accuracy from all iterations: %.2f%% (%.2f%%)' % (validation_score*100, validation_variance*100))
print('Total time for model fitting and cross validating:', (datetime.now() - startTimeModule))

Accuracy measurement from iteration 1 >>> 83.33%
Accuracy measurement from iteration 2 >>> 86.11%
Accuracy measurement from iteration 3 >>> 85.00%
Accuracy measurement from iteration 4 >>> 82.22%
Accuracy measurement from iteration 5 >>> 84.17%
Accuracy measurement from iteration 6 >>> 82.78%
Accuracy measurement from iteration 7 >>> 82.50%
Accuracy measurement from iteration 8 >>> 86.11%
Accuracy measurement from iteration 9 >>> 87.50%
Accuracy measurement from iteration 10 >>> 83.61%
Average model accuracy from all iterations: 84.33% (1.69%)
Total time for model fitting and cross validating: 0:02:32.864166


## 4.a) Alternate Model Two

In [47]:
# encode training and validation datasets
X_train, X_test = encode_data(training_docs, testing_docs, 'tfidf')
print('The shape of the encoded training dataset:', X_train.shape)
print('The shape of the encoded validation dataset:', X_test.shape)

The shape of the encoded training dataset: (1800, 25768)
The shape of the encoded validation dataset: (200, 25768)


In [48]:
# Initialize the default model and get a baseline result
startTimeModule = datetime.now()
results = list()
iteration = 0
cv = RepeatedKFold(n_splits=n_folds, n_repeats=n_iterations, random_state=seedNum)
for train_ix, val_ix in cv.split(X_train):
    feature_train, feature_validation = X_train[train_ix], X_train[val_ix]
    target_train, target_validation = y_train[train_ix], y_train[val_ix]
    reset_random(seedNum)
    alternate_model_2 = create_nn_model()
    alternate_model_2.fit(feature_train, target_train, epochs=default_epochs, batch_size=default_batch_size, verbose=0)
    model_metric = alternate_model_2.evaluate(feature_validation, target_validation, verbose=0)[1]
    iteration = iteration + 1
    print('Accuracy measurement from iteration %d >>> %.2f%%' % (iteration, model_metric*100))
    results.append(model_metric)
validation_score = np.mean(results)
validation_variance = np.std(results)
print('Average model accuracy from all iterations: %.2f%% (%.2f%%)' % (validation_score*100, validation_variance*100))
print('Total time for model fitting and cross validating:', (datetime.now() - startTimeModule))

Accuracy measurement from iteration 1 >>> 83.33%
Accuracy measurement from iteration 2 >>> 81.67%
Accuracy measurement from iteration 3 >>> 83.33%
Accuracy measurement from iteration 4 >>> 82.78%
Accuracy measurement from iteration 5 >>> 79.44%
Accuracy measurement from iteration 6 >>> 79.72%
Accuracy measurement from iteration 7 >>> 80.83%
Accuracy measurement from iteration 8 >>> 82.50%
Accuracy measurement from iteration 9 >>> 86.39%
Accuracy measurement from iteration 10 >>> 83.33%
Average model accuracy from all iterations: 82.33% (1.94%)
Total time for model fitting and cross validating: 0:02:34.509359


## 4.a) Alternate Model Three

In [49]:
# encode training and validation datasets
X_train, X_test = encode_data(training_docs, testing_docs, 'freq')
print('The shape of the encoded training dataset:', X_train.shape)
print('The shape of the encoded validation dataset:', X_test.shape)

The shape of the encoded training dataset: (1800, 25768)
The shape of the encoded validation dataset: (200, 25768)


In [50]:
# Initialize the default model and get a baseline result
startTimeModule = datetime.now()
results = list()
iteration = 0
cv = RepeatedKFold(n_splits=n_folds, n_repeats=n_iterations, random_state=seedNum)
for train_ix, val_ix in cv.split(X_train):
    feature_train, feature_validation = X_train[train_ix], X_train[val_ix]
    target_train, target_validation = y_train[train_ix], y_train[val_ix]
    reset_random(seedNum)
    alternate_model_3 = create_nn_model()
    alternate_model_3.fit(feature_train, target_train, epochs=default_epochs, batch_size=default_batch_size, verbose=0)
    model_metric = alternate_model_3.evaluate(feature_validation, target_validation, verbose=0)[1]
    iteration = iteration + 1
    print('Accuracy measurement from iteration %d >>> %.2f%%' % (iteration, model_metric*100))
    results.append(model_metric)
validation_score = np.mean(results)
validation_variance = np.std(results)
print('Average model accuracy from all iterations: %.2f%% (%.2f%%)' % (validation_score*100, validation_variance*100))
print('Total time for model fitting and cross validating:', (datetime.now() - startTimeModule))

Accuracy measurement from iteration 1 >>> 88.06%
Accuracy measurement from iteration 2 >>> 86.94%
Accuracy measurement from iteration 3 >>> 85.56%
Accuracy measurement from iteration 4 >>> 85.00%
Accuracy measurement from iteration 5 >>> 87.22%
Accuracy measurement from iteration 6 >>> 84.72%
Accuracy measurement from iteration 7 >>> 84.72%
Accuracy measurement from iteration 8 >>> 86.39%
Accuracy measurement from iteration 9 >>> 86.39%
Accuracy measurement from iteration 10 >>> 85.56%
Average model accuracy from all iterations: 86.06% (1.07%)
Total time for model fitting and cross validating: 0:02:34.788111


In [51]:
if notifyStatus: status_notify('(TensorFlow Text Classification) Task 4 - Evaluate and Optimize Models completed on ' + datetime.now().strftime('%A %B %d, %Y %I:%M:%S %p'))

# Task 5 - Finalize Model and Make Predictions

In [52]:
if notifyStatus: status_notify('(TensorFlow Text Classification) Task 5 - Finalize Model and Make Predictions has begun on ' + datetime.now().strftime('%A %B %d, %Y %I:%M:%S %p'))

In [53]:
# encode training and validation datasets
X_train, X_test = encode_data(training_docs, testing_docs, 'freq')
print('The shape of the encoded training dataset:', X_train.shape)
print('The shape of the encoded validation dataset:', X_test.shape)

The shape of the encoded training dataset: (1800, 25768)
The shape of the encoded validation dataset: (200, 25768)


In [54]:
# Create the final model for evaluating the test dataset
reset_random(seedNum)
final_model = create_nn_model()
final_model.fit(X_train, y_train, epochs=default_epochs, batch_size=default_batch_size, verbose=1)

Epoch 1/25
113/113 [==============================] - 1s 7ms/step - loss: 0.6720 - accuracy: 0.5828
Epoch 2/25
113/113 [==============================] - 1s 8ms/step - loss: 0.5542 - accuracy: 0.8000
Epoch 3/25
113/113 [==============================] - 1s 7ms/step - loss: 0.4412 - accuracy: 0.9150
Epoch 4/25
113/113 [==============================] - 1s 7ms/step - loss: 0.3592 - accuracy: 0.9639
Epoch 5/25
113/113 [==============================] - 1s 7ms/step - loss: 0.3007 - accuracy: 0.9806
Epoch 6/25
113/113 [==============================] - 1s 7ms/step - loss: 0.2557 - accuracy: 0.9894
Epoch 7/25
113/113 [==============================] - 1s 7ms/step - loss: 0.2209 - accuracy: 0.9944
Epoch 8/25
113/113 [==============================] - 1s 7ms/step - loss: 0.1929 - accuracy: 0.9967
Epoch 9/25
113/113 [==============================] - 1s 7ms/step - loss: 0.1699 - accuracy: 0.9983
Epoch 10/25
113/113 [==============================] - 1s 7ms/step - loss: 0.1512 - accuracy: 0.9983

In [55]:
# Summarize the final model
final_model.summary()

Model: "sequential_40"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_80 (Dense)             (None, 50)                1288450   
_________________________________________________________________
dense_81 (Dense)             (None, 1)                 51        
Total params: 1,288,501
Trainable params: 1,288,501
Non-trainable params: 0
_________________________________________________________________


In [56]:
# test_predictions = final_model.predict(X_test, batch_size=default_batch, verbose=1)
test_predictions = (final_model.predict(X_test) > 0.5).astype("int32").ravel()
print('Accuracy Score:', accuracy_score(y_test, test_predictions))
print(confusion_matrix(y_test, test_predictions))
print(classification_report(y_test, test_predictions))

Accuracy Score: 0.865
[[97  3]
 [24 76]]
              precision    recall  f1-score   support

           0       0.80      0.97      0.88       100
           1       0.96      0.76      0.85       100

    accuracy                           0.86       200
   macro avg       0.88      0.86      0.86       200
weighted avg       0.88      0.86      0.86       200



In [57]:
if notifyStatus: status_notify('(TensorFlow Text Classification) Task 5 - Finalize Model and Make Predictions completed on ' + datetime.now().strftime('%A %B %d, %Y %I:%M:%S %p'))

In [58]:
print ('Total time for the script:',(datetime.now() - startTimeScript))

Total time for the script: 0:12:08.151219
